# Задание 1
___
В предыдущих уроках вы уже познакомились с базовыми методами работы с таблицами (например, pivot) и встретились с иерархическими индексами (мультииндексами). Поначалу они могут выглядеть как что-то страшное и непонятное, от чего хочется поскорее избавиться. Но всё не так плохо! В pandas есть ряд полезных методов, которые упрощают работу с подобными индексами, а также позволяют с легкостью приводить данные к нужному формату. 

Начнем с методов stack и unstack, которые очень похожи на pivot(), и предназначены для работы с MultiIndex.

`stack`
Stack — помещает уровень столбцов в уровни индекса строк. Результирующий объект – Series.

![](https://ucarecdn.com/6cd4a57e-8add-4ff0-aca6-d62375f9d949/)

Например, создадим датафрейм из трех колонок:
```
np.random.seed(17)
df = pd.DataFrame({'col_1':['item_1', 'item_2', 'item_3', 'item_4', 'item_5'], 
                   'col_2': np.random.randint(0, 10, 5), 
                   'col_3': np.random.randint(0, 10, 5)})
df

     col_1  col_2  col_3
0   item_1      1      6
1   item_2      6      4
2   item_3      6      7
3   item_4      9      4
4   item_5      0      7
```
Применяем .stack():
```
df_stacked = df.stack()
df_stacked

0  col_1    item_1
   col_2         1
   col_3         6
1  col_1    item_2
   col_2         6
   col_3         4
2  col_1    item_3
   col_2         6
   col_3         7
3  col_1    item_4
   col_2         9
   col_3         4
4  col_1    item_5
   col_2         0
   col_3         7
dtype: object
```
Индексам также можно присвоить названия:
```
df_stacked.index.names = ['id', 'column']
df_stacked

id  column
0   col_id    item_1
    col_2          7
    col_3          0
1   col_id    item_2
    col_2          4
    col_3          9
2   col_id    item_3
    col_2          4
    col_3          7
3   col_id    item_4
    col_2          5
    col_3          5
4   col_id    item_5
    col_2          8
    col_3          3
dtype: object
```
В качестве аргументов stack можно передать два параметра.

level – отвечает за уровень, по которому будет проведена стыковка
dropna – нужно ли убрать ряды с пропущенными значениями 
Посмотрим как это работает на примере следующего датафрейма о весе и росте котиков:
```
                weight          height
        old_kg  new_kg  old_cm  new_cm
Persik     NaN     3.4     NaN    26.0
Barsik     3.0     4.1    25.0    30.0
```
Сначала применяем метод stack без указания аргументов. По умолчанию стыковка происходит по уровню -1, а ряды с пропущенными значениями удаляются. Теперь в датафрейме есть две колонки – height и weight и два уровня индексов. 
```
df.stack()

                height  weight
Persik  new_cm    26.0     NaN
        new_kg     NaN     3.4
Barsik  new_cm    30.0     NaN
        new_kg     NaN     4.1
        old_cm    25.0     NaN
        old_kg     NaN     3.0
```
Как бы выглядела табличка с пропущенными значениями? Можно заметить, что в случае dropna=False, для Персика появились еще две строки со старыми параметрами роста и веса (old_cm и old_kg), которые полностью заполнены NaN.
```
df.stack(dropna=False)

                height  weight
Persik  new_cm    26.0     NaN
        new_kg     NaN     3.4
        old_cm     NaN     NaN
        old_kg     NaN     NaN
Barsik  new_cm    30.0     NaN
        new_kg     NaN     4.1
        old_cm    25.0     NaN
        old_kg     NaN     3.0
```
Теперь проведем стыковку по нулевому уровню. В качестве столбцов выступают new_cm, new_kg, old_cm и old_kg, а в индексах остались имена животных и рост/вес.
```
                new_cm  new_kg  old_cm  old_kg
Persik  height    26.0     NaN	   NaN     NaN
        weight     NaN     3.4     NaN     NaN
Barsik  height    30.0     NaN    25.0     NaN
        weight     NaN     4.1     NaN     3.0

xs
```
Значения конкретного уровня индексов можно получить используя метод .xs(), передав ему интересующее нас значение индекса и уровень. Например, чтобы вывести все значения для height, в аргументах нужно указать само значение и название столбца с индексами интересующего нас уровня, в данном случае — param.

Применяем stack и присваиваем уровням индекса названия:
```
df_stacked_2 = df.stack([0,1])
df_stacked_2.index.names = ['name', 'param', 'param_type']  # присваиваем индексам названия
df_stacked_2

name    param    param_type
Persik  height   new_cm      26.0
        weight   new_kg       3.4
Barsik  height   new_cm      30.0
                 old_cm      25.0
        weight   new_kg       4.1
                 old_kg       3.0
dtype: float64
```
Достаем все значения роста height из уровня индексов param с помощью xs: 
```
df_stacked_2.xs('height', level='param')

name    param_type
Persik  new_cm        26.0
Barsik  new_cm        30.0
        old_cm        25.0
dtype: float64
```
Попробуем достать значения из колонок исходного датафрейма. Для этого необходимо указать axis=1, далее – уровень и ключ, т.е. название интересующего нас уровня. Например, возьмем старый вес Персика и Барсика: 
```
df

                weight          height    # level 0
        old_kg  new_kg  old_cm  new_cm    # level 1
Persik     NaN     3.4     NaN    26.0
Barsik     3.0     4.1    25.0    30.0

df.xs(axis=1, level=1, key='old_kg')

        weight
Persik     NaN
Barsik     3.0
```
Старый и новый:
```
df.xs(axis=1, key='weight')  # в данном случае можно не указывать level, т.к. по умолчанию level=0

        old_kg  new_kg
Persik     NaN     3.4
Barsik     3.0     4.1
```
Note: Чтобы получить значения из исходного датафрейма, можно также передать кортеж из уровней:
```
df[('weight', 'old_kg')]

Persik    NaN
Barsik    3.0
Name: (weight, old_kg), dtype: float64
```

In [4]:
import numpy as np
import pandas as pd

In [8]:
url = 'https://stepik.org/media/attachments/lesson/363873/nyc.csv.zip'

In [9]:
nyc = pd.read_csv(url, compression='zip') #исходный дата-фрейм

In [10]:
def haversine(fi_1, lam_1, fi_2, lam_2, radius=6371):
    # перевод в градусов радианы
    fi_1, lam_1, fi_2, lam_2 = [np.radians(x) for x in (fi_1, lam_1, fi_2, lam_2)]
    
    # выводим результаты
    return 2 * radius * np.arcsin(((np.sin((fi_2 - fi_1) / 2)) ** 2 + np.cos(fi_1) * np.cos(fi_2) * (np.sin((lam_2 - lam_1) / 2)) ** 2) ** 0.5)

In [12]:
nyc['distance'] = haversine(nyc['pickup_latitude'].values, nyc['pickup_longitude'].values, nyc['dropoff_latitude'].values, nyc['dropoff_longitude'].values)

In [13]:
nyc.head()

,key,fare_amount,pickup_datetime,pickup_longitude,pickup_latitude,dropoff_longitude,dropoff_latitude,passenger_count,distance
0,2009-06-15 17:26:21.0000001,4.5,2009-06-15 17:26:21 UTC,-73.844311,40.721319,-73.841610,40.712278,1,1.030764
1,2010-01-05 16:52:16.0000002,16.9,2010-01-05 16:52:16 UTC,-74.016048,40.711303,-73.979268,40.782004,1,8.450134
2,2011-08-18 00:35:00.00000049,5.7,2011-08-18 00:35:00 UTC,-73.982738,40.761270,-73.991242,40.750562,2,1.389525
3,2012-04-21 04:30:42.0000001,7.7,2012-04-21 04:30:42 UTC,-73.987130,40.733143,-73.991567,40.758092,1,2.799270
4,2010-03-09 07:51:00.000000135,5.3,2010-03-09 07:51:00 UTC,-73.968095,40.768008,-73.956655,40.783762,1,1.999157


In [14]:
nyc.sort_values('key', inplace=True)

In [15]:
nyc.head()

,key,fare_amount,pickup_datetime,pickup_longitude,pickup_latitude,dropoff_longitude,dropoff_latitude,passenger_count,distance
12739,2009-01-01 01:31:49.0000003,8.6,2009-01-01 01:31:49 UTC,-73.994192,40.720077,-73.993356,40.742642,1,2.510102
17546,2009-01-01 02:05:37.0000004,11.0,2009-01-01 02:05:37 UTC,-73.978433,40.744781,-74.004713,40.734328,1,2.500651
25687,2009-01-01 02:07:49.0000001,17.8,2009-01-01 02:07:49 UTC,-73.984291,40.667851,-74.006015,40.735481,3,7.739881
8684,2009-01-01 02:51:52.0000002,10.2,2009-01-01 02:51:52 UTC,-73.956172,40.771965,-73.991027,40.751035,2,3.746203
27126,2009-01-01 03:31:36.0000001,15.4,2009-01-01 03:31:36 UTC,-73.980325,40.734579,-74.014711,40.717868,3,3.442244


In [22]:
%%timeit
nyc.query('key >= "2009-01-01 01:31:49.0000003"')

15.3 ms ± 809 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)


In [18]:
rr = nyc.set_index('key')

In [25]:
%%timeit
rr.loc["2009-01-01 01:31:49.0000003":]
# по индексам отбираем быстрее

35.3 µs ± 2.44 µs per loop (mean ± std. dev. of 7 runs, 10000 loops each)


In [26]:
df = pd.DataFrame(
                    {
                        'a' : [1, 1, 3],
                        'b' : [20, 20, 30],
                        'c' : [1, 20, 100]
                    })
df

,a,b,c
0,1,20,1
1,1,20,20
2,3,30,100


In [29]:
indexed = df.groupby(['a', 'b']) \
    .agg({'c': 'count'})
indexed

,,c
a,b,
1,20,2
3,30,1


In [30]:
df.groupby(['a', 'b']) \
    .agg({'c': 'count'}) \
    .index

MultiIndex([(1, 20),
            (3, 30)],
           names=['a', 'b'])

In [31]:
df.groupby(['a', 'b'], as_index=False) \
    .agg({'c': 'count'})

,a,b,c
0,1,20,2
1,3,30,1


In [32]:
indexed.index.set_names(['first', 'second'], inplace=True)
indexed

,,c
first,second,
1,20,2
3,30,1


In [33]:
indexed.index.names = ['qst', '2nd']
indexed

,,c
qst,2nd,
1,20,2
3,30,1


In [34]:
df.set_index(['a', 'b'])

c
a b      
1 20    1
  20   20
3 30  100

In [35]:
df.set_index(['a', 'b'], inplace=True)
df

c
a b      
1 20    1
  20   20
3 30  100

In [36]:
df.loc[(1, 20)]

c
a b     
1 20   1
  20  20

# Задание 2
___
unstack
Unstack (расстыковка) — операция, противоположная stack, которая помещает уровень индекса строк в уровень оси столбцов.

![](https://ucarecdn.com/3a5b2090-9639-4b29-874d-577de5ff591a/)

Возьмем датафрейм из предыдущего стэпа:

df_stacked_2

name    param    param_type
Persik  height   new_cm      26.0
        weight   new_kg       3.4
Barsik  height   new_cm      30.0
                 old_cm      25.0
        weight   new_kg       4.1
                 old_kg       3.0
dtype: float64

Применяем:

df_stacked_2.unstack(level=1)

        param      height   weight
name    param_type		
Persik      new_cm   26.0      NaN
            new_kg    NaN      3.4
Barsik      new_cm   30.0      NaN
            new_kg    NaN      4.1
            old_cm   25.0      NaN
            old_kg    NaN      3.0

Один из аргументов метода — level, отвечает за уровень, по которому будет проведена расстыковка. Можно передать как число, так и название уровня индексов, если таковое имеется. В данном случае, 0 или "name", 1 или "param", 2 или "param_type".

df_stacked_2.unstack(level='name')

        name        Persik   Barsik
param   param_type		
height	new_cm        26.0     30.0
        old_cm         NaN     25.0
weight  new_kg         3.4      4.1
        old_kg         NaN      3.0

![](https://ucarecdn.com/39ea5824-8734-4b48-8999-3fc8e05511e0/)
![](https://ucarecdn.com/35319fb8-72ba-4429-81b6-0aa334c6006e/)`


Для того, чтобы сделать расстыковку по нескольким уровням, необходимо передать список, состоящий либо только из чисел, либо только из названий. Поместим параметры в столбцы, а имена оставим в индексах:

df_stacked_2.unstack(level=['param', 'param_type'])

param       height  weight  height  weight
param_type  new_cm  new_kg  old_cm  old_kg
      name				
    Persik    26.0     3.4     NaN     NaN
    Barsik    30.0     4.1    25.0     3.0

Вопрос: в чем же разница между stack и unstack?

In [41]:
df

c
a b      
1 20    1
  20   20
3 30  100

# Задание 3
___
Теперь задача. В этот раз никаких такси и товаров, потренируемся на характеристиках покемонов! В исходном датасете мультииндексов нет, поэтому будем создавать их сами. [Данные](https://stepik.org/media/attachments/lesson/363874/Pokemon.csv) сохранены в переменную pokemon.

Сначала измените названия исходных столбцов:

+ пробелы и точки нужно заменить на "_" (напр. Sp. Atk → sp_atk).
+ приведите все названия к нижнему регистру
+ колонку "#" переименовать в "id"

Полученные результаты запишите в исходный датафрейм pokemon.

Затем сгруппируйте данные по поколению покемонов (generation), и с помощью value_counts() посчитайте, сколько в каком поколении легендарных покемонов (legendary), а также сколько в этих поколениях нелегендарных покемонов. Полученный объект приведите к формату датафрейма (.to_frame()) и сохраните в legends.

In [37]:
url = 'https://stepik.org/media/attachments/lesson/363874/Pokemon.csv'

In [46]:
pokemon = pd.read_csv(url)

In [50]:
pokemon.rename(columns=lambda x: x.replace(' .', '_')
                                   .replace(' ', '_')
                                   .lower(), 
               inplace=True)

In [52]:
pokemon.rename(columns={'#' : 'id'}, inplace=True)

In [54]:
pokemon.head()

,id,name,type_1,type_2,total,hp,attack,defense,sp._atk,sp._def,speed,generation,legendary
0,1,Bulbasaur,Grass,Poison,318,45,49,49,65,65,45,1,False
1,2,Ivysaur,Grass,Poison,405,60,62,63,80,80,60,1,False
2,3,Venusaur,Grass,Poison,525,80,82,83,100,100,80,1,False
3,3,VenusaurMega Venusaur,Grass,Poison,625,80,100,123,122,120,80,1,False
4,4,Charmander,Fire,NaN,309,39,52,43,60,50,65,1,False


In [60]:
legends = pokemon.groupby('generation') \
                    .legendary \
                    .value_counts() \
                    .to_frame()

In [61]:
legends

legendary
generation legendary           
1          False            160
           True               6
2          False            101
           True               5
3          False            142
           True              18
4          False            108
           True              13
5          False            150
           True              15
6          False             74
           True               8

# Задание 4
___
Как вы могли заметить,  есть только одна колонка со значениями, которая называется `legendary`, а в качестве индекса используются две колонки — `generation` и `legendary`. Два одинаковых названия — не очень хорошо, поэтому необходимо переименовать колонку `legendary` в `legendary_count`. 

Используйте датафрейм **legends**, полученный на предыдущем шаге, и измените в нём название столбца, перезаписав его в ту же переменную. Затем используйте `unstack`, чтобы поместить уровень индекса `legendary` в уровень оси столбцов. Иными словами, должно получиться две колонки – False & True. Результат сохраните в `legends_unstacked`.

Пример ожидаемого формата названия колонок:
![](https://ucarecdn.com/19dfb6dd-d2f1-43d3-99a3-ba1100d409ec/)

In [65]:
legends.rename(columns={
    'legendary':'legendary_count'
}, inplace=True)

In [68]:
legends_unstacked = legends.unstack()

In [69]:
legends_unstacked

legendary_count      
legendary            False True 
generation                      
1                      160     6
2                      101     5
3                      142    18
4                      108    13
5                      150    15
6                       74     8

# Задание 5
___
Немного усложним задачу. Теперь попробуем узнать, среди каких типов покемонов и какого поколения больше всего легендарных.

Сгруппируйте датасет `pokemon` по переменным `generation` и `type_1`, посчитайте количество легендарных покемонов внутри групп. Приведите данные в формат датафрейма, а затем используйте `unstack()`. В качестве ответа выберите вид и поколение покемона, среди которых больше всего легендарных.

Hint: обратиться к колонке с мультииндексом можно с помощью `.loc[:,('legendary', True)]`

In [70]:
pokemon.head()

,id,name,type_1,type_2,total,hp,attack,defense,sp._atk,sp._def,speed,generation,legendary
0,1,Bulbasaur,Grass,Poison,318,45,49,49,65,65,45,1,False
1,2,Ivysaur,Grass,Poison,405,60,62,63,80,80,60,1,False
2,3,Venusaur,Grass,Poison,525,80,82,83,100,100,80,1,False
3,3,VenusaurMega Venusaur,Grass,Poison,625,80,100,123,122,120,80,1,False
4,4,Charmander,Fire,NaN,309,39,52,43,60,50,65,1,False


In [76]:
pokemon.groupby(['generation', 'type_1']) \
        .legendary \
        .value_counts() \
        .unstack() \
        .idxmax()

legendary
False     (1, Water)
True     (3, Dragon)
dtype: object

# Задание 6
___
Широкий и длинный формат
melt
С помощью метода melt можно "расплавить" данные и привести их к длинному формату. Так, одна или несколько колонок помещаются в качестве идентификационных переменных, а остальные столбцы считаются измеряемыми переменными. Их названия и значения помещаются в колонки variable и value.

![](https://ucarecdn.com/6b94af41-b3a8-48ee-8d0a-b39dd4ea0c0b/)

Для изменения названий полученных столбцов используются параметры var_name и value_name.

Посмотрим на примере маленького датасета с характеристиками:
```
df3 = pd.DataFrame({'name': ['Persik', 'Brownie'], 'type': ['cat', 'dog'],
                    'color': ['ginger', 'white'], 'height': [17, 30], 
                    'weight': [3.4, 4.3]})
df3

      name type   color  height  weight
0   Persik  cat  ginger      17     3.4
1  Brownie  dog   white      30     4.3
```
Расплавляем! Если не указать колонки, которые нужно использовать в качестве идентификаторов, то названия всех столбцов помещаются в variable, а соответствующие в value.
```
df3.melt().head()

  variable    value
0     name   Persik
1     name  Brownie
2     type      cat
3     type      dog
4    color   ginger
```
Используем имена в качестве идентификатора:
```
df3.melt(id_vars='name').head()

      name variable   value
0   Persik     type     cat
1  Brownie     type     dog
2   Persik    color  ginger
3  Brownie    color   white
4   Persik   height      17
```
Для изменения названий полученных столбцов используются параметры var_name и value_name. Например, передаем в качестве id_vars имена, для значений (value_vars) используем только три колонки и изменяем названия новых колонок:
```
df3.melt(id_vars=['name'], value_vars=['type', 'color', 'height'], 
         var_name='characteristics', value_name='value')

      name characteristics   value
0   Persik            type     cat
1  Brownie            type     dog
2   Persik           color  ginger
3  Brownie           color   white
4   Persik          height      17
5  Brownie          height      30
```
wide_to_long
Еще один вариант для перевода данных из широкого формата в длинный — pd.wide_to_long(). 

Предположим, мы собрали побольше данных о котике Персике и пёсике Брауни, и добавили данные о весе и росте уже за два года:
```
      name type  AvgHeight_2019  AvgHeight_2020  AvgWeight_2019  AvgWeight_2020   color 
0   Persik  cat       17.077963       17.134233             3.4          3.5545  ginger
1  Brownie  dog       30.673324       30.674466             4.3          4.5716   white    
```
Посмотрим на аргументы функции более подробно.

data — датафрейм
stubnames — части названий переменных, которые мы хотим преобразовать из широкого формата в длинный
i — переменные, которые не трансформируются, и в результате помещаются в индексы
j — имя новой переменной
sep — разделитель (между параметром и значением)
В данном случае у нас есть две общих характеристики, отвечающих за рост и вес в конкретный год. Названия соответствующих переменных состоят из AvgHeight / AvgWeight и года, поэтому в stubnames мы передаем список параметров (вес и рост), а оставшаяся часть названия (2018, 2019) будет использована в качестве значений новой переменной year. Столбцы type и name помещаем в индексы, а параметр color оставляем обычной колонкой.
```
lng = pd.wide_to_long(df4, ['AvgHeight', 'AvgWeight'], i=['type', 'name'], j='year', sep='_')
lng

                    color  AvgHeight  AvgWeight
type name    year                              
cat  Persik  2019  ginger  17.077963     3.4000
             2020  ginger  17.134233     3.5545
dog  Brownie 2019   white  30.673324     4.3000
             2020   white  30.674466     4.5716
```
А теперь возвращаем всё обратно к широкому формату:
```
wd = lng.unstack(level='year')
wd.columns = ['_'.join(map(str, col)) for col in wd.columns]  # соединяем названия
wd.drop('color_2019', inplace=True, axis=1)  # убираем лишнюю колонку
wd = wd.rename(columns={'color_2020': 'color'})  # исправляем название
wd.reset_index() # избавляемся от мультииндекса

  type     name   color  AvgHeight_2019  AvgHeight_2020  AvgWeight_2019  AvgWeight_2020
0  cat   Persik  ginger       17.077963       17.134233             3.4          3.5545   
1  dog  Brownie   white       30.673324       30.674466             4.3          4.5716 
```

In [77]:
df3 = pd.DataFrame({'name': ['Persik', 'Brownie'], 'type': ['cat', 'dog'],
                    'color': ['ginger', 'white'], 'height': [17, 30], 
                    'weight': [3.4, 4.3]})
df3

,name,type,color,height,weight
0,Persik,cat,ginger,17,3.4
1,Brownie,dog,white,30,4.3


In [78]:
df3.melt()

,variable,value
0,name,Persik
1,name,Brownie
2,type,cat
3,type,dog
4,color,ginger
5,color,white
6,height,17
7,height,30
8,weight,3.4
9,weight,4.3


In [83]:
df3.melt(id_vars=['name', 'type'], 
         var_name='parameter', 
        value_name='val', 
        value_vars=['height', 'weight'])

,name,type,parameter,val
0,Persik,cat,height,17.0
1,Brownie,dog,height,30.0
2,Persik,cat,weight,3.4
3,Brownie,dog,weight,4.3


In [84]:
df3.melt(id_vars=['name'], 
         var_name='parameter', 
        value_name='val')

,name,parameter,val
0,Persik,type,cat
1,Brownie,type,dog
2,Persik,color,ginger
3,Brownie,color,white
4,Persik,height,17
5,Brownie,height,30
6,Persik,weight,3.4
7,Brownie,weight,4.3


# Задание 7
___
Преобразуйте представленные данные в длинный формат и запишите в переменную avocado_agg_long. В качестве индекса используйте type
```
avocado_agg = pd.DataFrame({'type' : ['conventional', 'organic'],
                            'AvgPrice_2015' : [1.077963, 1.673324],
                            'AvgPrice_2016' : [1.105595, 1.571684],
                            'AvgPrice_2017' : [1.294888, 1.735521],
                            'AvgPrice_2018' : [1.127886, 1.567176],
                            })
avocado_agg
```


Может пригодиться:

wide_to_long
Агрегированные данные сохранены в avocado_agg.

In [86]:
avocado_agg = pd.DataFrame({'type' : ['conventional', 'organic'],
                            'AvgPrice_2015' : [1.077963, 1.673324],
                            'AvgPrice_2016' : [1.105595, 1.571684],
                            'AvgPrice_2017' : [1.294888, 1.735521],
                            'AvgPrice_2018' : [1.127886, 1.567176],
                            })
avocado_agg

,type,AvgPrice_2015,AvgPrice_2016,AvgPrice_2017,AvgPrice_2018
0,conventional,1.077963,1.105595,1.294888,1.127886
1,organic,1.673324,1.571684,1.735521,1.567176


In [90]:
avocado_agg_long = pd.wide_to_long(avocado_agg, stubnames='AvgPrice', i='type', sep='_', j='year')

In [91]:
avocado_agg_long

,,AvgPrice
type,year,
conventional,2015,1.077963
organic,2015,1.673324
conventional,2016,1.105595
organic,2016,1.571684
conventional,2017,1.294888
organic,2017,1.735521
conventional,2018,1.127886
organic,2018,1.567176


# Заданеи 8
___
⭐️ Имеется набор [данных](https://stepik.org/media/attachments/lesson/363874/superheroes_power_matrix.csv) о супергероях в широком формате. В первой колонке Name находятся их имена, а остальные 167 столбцов — различные характеристики (суперсилы), принимающие значение либо True, либо False. Давайте преобразуем датафрейм так, чтобы "собрать" эти признаки в один столбец под названием superpower, а в качестве значений поместить туда списки имеющихся у того или иного героя суперсил. 

То есть привести в следующий формат:

           Name                                         superpower
568  Spider-Man  [Reflexes, Animal Oriented Powers, Danger Sens...

Сначала приведите данные к длинному формату, где единственным идентификатором будет имя героя Name. Новый столбец с названиями суперсил переименуйте в superpower. Полученный датафрейм запишите в superheroes_long.

Для того, чтобы соединить силы в списки и привести данные в желаемую форму, отфильтруйте колонку value так, чтобы остались строки только со значением True. Сгруппируйте датасет по Name, после чего возьмите столбец superpower и используйте .apply(list). Результат сохраните в superheroes_powers. Обратите внимание, что результирующий объект тоже должен быть датафреймом, а колонки должно быть всего две — Name и superpower.

Данные записаны в superheroes

In [92]:
url = 'https://stepik.org/media/attachments/lesson/363874/superheroes_power_matrix.csv'

In [108]:
superheros = pd.read_csv(url)

In [109]:
superheros.head()

,Name,Agility,Accelerated Healing,Lantern Power Ring,Dimensional Awareness,Cold Resistance,Durability,Stealth,Energy Absorption,Flight,...,Web Creation,Reality Warping,Odin Force,Symbiote Costume,Speed Force,Phoenix Force,Molecular Dissipation,Vision - Cryo,Omnipresent,Omniscient
0,3-D Man,True,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
1,A-Bomb,False,True,False,False,False,True,False,False,False,...,False,False,False,False,False,False,False,False,False,False
2,Abe Sapien,True,True,False,False,True,True,False,False,False,...,False,False,False,False,False,False,False,False,False,False
3,Abin Sur,False,False,True,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
4,Abomination,False,True,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False


In [110]:
superheroes_long = superheros.melt(id_vars=['Name'], var_name='superpower')

In [111]:
superheroes_long.head()

,Name,superpower,value
0,3-D Man,Agility,True
1,A-Bomb,Agility,False
2,Abe Sapien,Agility,True
3,Abin Sur,Agility,False
4,Abomination,Agility,False


In [112]:
superheroes_powers = superheroes_long.query('value == True') \
                                        .groupby('Name') \
                                        .superpower \
                                        .apply(list) \
                                        .to_frame() \
                                        .reset_index()

In [113]:
superheroes_powers

,Name,superpower
0,3-D Man,"[Agility, Super Strength, Stamina, Super Speed]"
1,A-Bomb,"[Accelerated Healing, Durability, Longevity, S..."
2,Abe Sapien,"[Agility, Accelerated Healing, Cold Resistance..."
3,Abin Sur,[Lantern Power Ring]
4,Abomination,"[Accelerated Healing, Intelligence, Super Stre..."
...,...,...
662,Yellowjacket II,"[Flight, Energy Blasts, Size Changing]"
663,Ymir,"[Cold Resistance, Durability, Longevity, Super..."
664,Yoda,"[Agility, Stealth, Danger Sense, Marksmanship,..."
665,Zatanna,"[Cryokinesis, Telepathy, Magic, Fire Control, ..."


# Задание 9
___
explode
Одно из нововведений в pandas версии 0.25.0 — метод explode(). Сначала создадим датафрейм из двух столбцов: колонку B заполним единичками, а в А запишем следующие элементы:

в две ячейки – списки, состоящие из нескольких элементов
пустой список
'kitten'
df = pd.DataFrame({'A': [[1, 2, 3], 
                         'kitten', 
                         [], 
                         ['kitten', 'puppy']], 
                   'B': 1})
df

               A   B
0      [1, 2, 3]   1
1         kitten   1
2              []  1
3 [kitten, puppy]  1

Такой формат данных в ячейках не очень удобен для дальнейшей работы. Например, как нам посчитать, сколько раз встретилось то или иное значение в А?

Как раз здесь нам поможет explode. Метод преобразовывает каждый элемент списка в отдельный ряд, при этом сами индексы строк дублируются. На вход необходимо передать либо одну колонку, либо их список.

df.explode('A')

       A   B
0      1   1
0      2   1
0      3   1
1 kitten   1
2    NaN   1
3 kitten   1
3  puppy   1

Посчитаем, сколько раз встречаются те или иные значения:

df.explode('A').A.value_counts().to_frame(name='count')  # переименовываем "A" в "count"

       count
kitten     2
puppy      1
3          1
2          1
1          1